In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForCausalLM, 
    TrainingArguments, BitsAndBytesConfig, DataCollatorForLanguageModeling
)
from sklearn.model_selection import train_test_split
from datasets import Dataset as HFDataset
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from trl import SFTTrainer
import re
import gc
import os
import warnings
warnings.filterwarnings('ignore')

d:\ML_PROJECTS\TUESDAY\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomCompletionOnlyCollator: 
    """
    Custom data collator that only computes loss on the completion/response part.
    This replaces DataCollatorForCompletionOnlyLM for older trl versions.
    """
    def __init__(self, tokenizer, response_template="<|assistant|>", mlm=False, max_length=None):
        self.tokenizer = tokenizer
        self.response_template = response_template
        self.response_template_ids = tokenizer.encode(
            response_template, add_special_tokens=False
        )
        self.mlm = mlm
        self.max_length = max_length
    
    def __call__(self, examples):
        # Handle both list of dicts and dict of lists
        if isinstance(examples, list):
            if examples and isinstance(examples[0], dict) and "input_ids" not in examples[0]:
                texts = []
                for example in examples:
                    if isinstance(example, dict):
                        if "text" in example:
                            texts.append(example["text"] )
                        else:
                            raise ValueError("Example dictionary missing 'text' field required for tokenization.")
                    else:
                        texts.append(str(example))
                batch = self.tokenizer(
                    texts,
                    padding=True,
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt"
                )
            else:
                batch = self.tokenizer.pad(
                    examples,
                    padding=True,
                    return_tensors="pt"
                )
        else:
            batch = examples
        
        labels = batch["input_ids"].clone()
        
        # For each sequence, mask everything before the response template
        for i in range(len(labels)):
            input_ids = batch["input_ids"][i].tolist()
            
            # Find the response template position
            response_start = self._find_response_start(input_ids)
            
            if response_start != -1:
                # Mask everything before the response (set to -100)
                labels[i, :response_start] = -100
            
            # Also mask padding tokens
            if self.tokenizer.pad_token_id is not None: 
                labels[i, labels[i] == self.tokenizer.pad_token_id] = -100
        
        batch["labels"] = labels
        return batch
    
    def _find_response_start(self, input_ids):
        """Find where the response template starts in the input."""
        template_len = len(self.response_template_ids)
        for i in range(len(input_ids) - template_len + 1):
            if input_ids[i:i + template_len] == self.response_template_ids:
                return i + template_len
        return -1

In [3]:
class Config:
    # Paths - FIXED:  no space in filename
    DATA_PATH = r"D:\ML_PROJECTS\TUESDAY\Tuesday_bot\cleaned.csv"
    OUTPUT_DIR = "./models"
    
    # Model names
    CLASSIFIER_MODEL = "distilbert-base-uncased"
    POLICY_MODEL = "microsoft/Phi-3.5-mini-instruct"
    
    # Training hyperparameters - optimized for 12GB VRAM
    CLASSIFIER_BATCH_SIZE = 16
    CLASSIFIER_EPOCHS = 5
    CLASSIFIER_LR = 2e-5
    CLASSIFIER_MAX_LEN = 128
    
    POLICY_BATCH_SIZE = 1
    POLICY_GRAD_ACCUM = 8
    POLICY_EPOCHS = 3
    POLICY_LR = 1e-4
    POLICY_MAX_LEN = 512
    
    # LoRA settings
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.05
    
    # Device
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = Config()
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

In [4]:
def clear_gpu_memory():
    """Aggressively clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda. synchronize()

def print_gpu_memory():
    """Print current GPU memory usage."""
    if torch. cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        print(f"GPU Memory:  {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

In [5]:
class ImprovedMentalStateModel(nn.Module):
    """
    Improved multi-task classifier with:
    - Task-specific attention
    - Residual connections
    - Better regularization
    """
    def __init__(self, num_emotions, num_intents, num_risk, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        h = self.encoder.config.hidden_size
        
        # Store config for reconstruction
        self.num_emotions = num_emotions
        self.num_intents = num_intents
        self.num_risk = num_risk
        self.model_name = model_name
        
        # Shared layers with residual
        self.shared1 = nn.Linear(h, h)
        self.shared2 = nn.Linear(h, h)
        self.layer_norm = nn.LayerNorm(h)
        self.dropout = nn.Dropout(0.3)
        
        # Task-specific heads with intermediate layers
        self.emotion_hidden = nn.Linear(h, h // 2)
        self.emotion_head = nn.Linear(h // 2, num_emotions)
        
        self.intent_hidden = nn.Linear(h, h // 4)
        self.intent_head = nn.Linear(h // 4, num_intents)
        
        self.risk_hidden = nn.Linear(h, h // 4)
        self.risk_head = nn.Linear(h // 4, num_risk)
        
        self.intensity_hidden = nn.Linear(h, h // 4)
        self.intensity_head = nn.Linear(h // 4, 1)
        
        # Confidence estimation
        self.confidence_head = nn.Linear(h, 1)
        
    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids, attention_mask=attention_mask)
        
        # Attention-weighted pooling
        hidden = out.last_hidden_state
        attention_weights = attention_mask.unsqueeze(-1).float()
        pooled = (hidden * attention_weights).sum(dim=1) / attention_weights.sum(dim=1).clamp(min=1e-9)
        
        # Shared representation with residual
        z = self.dropout(F.gelu(self.shared1(pooled)))
        z = self.layer_norm(z + self.dropout(F.gelu(self.shared2(z))))
        
        # Task-specific outputs
        emotion_h = self.dropout(F.gelu(self.emotion_hidden(z)))
        intent_h = self.dropout(F.gelu(self.intent_hidden(z)))
        risk_h = self.dropout(F.gelu(self.risk_hidden(z)))
        intensity_h = self.dropout(F.gelu(self.intensity_hidden(z)))
        
        return {
            "emotion": self.emotion_head(emotion_h),
            "intent": self.intent_head(intent_h),
            "risk": self.risk_head(risk_h),
            "intensity": torch.sigmoid(self.intensity_head(intensity_h)).squeeze(-1),
            "confidence": torch.sigmoid(self.confidence_head(z)).squeeze(-1),
            "semantic_vector": z
        }

In [6]:
print("=" * 80)
print("PART 1: Data Preprocessing")
print("=" * 80)

df = pd.read_csv(config.DATA_PATH)

def clean_user_text(text):
    """Clean and normalize user text."""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r'^\s*customer\s*:\s*', '', text, flags=re.I)
    text = re.split(r'\bagent\s*:', text, flags=re.I)[0]
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df["clf_text"] = df["empathetic_dialogues"].apply(clean_user_text)

# Enhanced filtering
df = df[df["labels"]. notna()]
df = df[df["labels"]. str.strip() != ""]
df = df[df["labels"].str.split().str.len() >= 3]
df = df[df["clf_text"].str.len() >= 10]
df = df. reset_index(drop=True)

# Emotion filtering
emotion_counts = df["emotion"].value_counts()
valid_emotions = emotion_counts[emotion_counts >= 15].index
df = df[df["emotion"]. isin(valid_emotions)].reset_index(drop=True)

# Create emotion labels
df["label"] = df["emotion"].astype("category").cat.codes
id2label = dict(enumerate(df["emotion"].astype("category").cat.categories))
label2id = {v:  k for k, v in id2label.items()}
NUM_EMOTIONS = len(id2label)

print(f"Dataset size: {len(df)}")
print(f"Number of emotions: {NUM_EMOTIONS}")
print(f"Emotions:  {list(id2label.values())}")

# Enhanced intensity mapping
INTENSITY_MAP = {
    "sad": 0.8, "angry": 0.9, "anxious": 0.85, "fear": 0.9,
    "lonely": 0.8, "depressed": 0.95, "hopeless": 0.95,
    "neutral": 0.3, "content": 0.25,
    "happy": 0.2, "excited": 0.4, "grateful": 0.3, "proud": 0.4,
    "surprised": 0.5, "confused": 0.6,
}
df["intensity"] = df["emotion"].apply(lambda e: INTENSITY_MAP. get(e. lower(), 0.5))

# Enhanced intent inference
def infer_intent(text):
    t = text.lower()
    if any(x in t for x in ["what should i", "how do i", "how can i", "can you help", 
                            "any advice", "tips", "suggestions", "recommend", "what would you"]):
        return "advice"
    if any(x in t for x in ["just wanted to", "needed to vent", "i feel like", "honestly", 
                            "ngl", "lowkey", "i just", "ugh", "i hate", "so tired of"]):
        return "venting"
    return "validation"

INTENT_MAP = {"venting": 0, "advice": 1, "validation": 2}
df["intent_id"] = df["clf_text"]. apply(lambda x: INTENT_MAP[infer_intent(x)])
NUM_INTENTS = len(INTENT_MAP)

# Enhanced risk inference
def infer_risk(text):
    t = text.lower()
    high_risk_keywords = [
        "kill myself", "end it all", "can't go on", "want to die", "suicide",
        "better off dead", "end my life", "hurt myself", "self harm", "cutting"
    ]
    if any(x in t for x in high_risk_keywords):
        return 2
    medium_risk_keywords = [
        "hopeless", "worthless", "give up", "no point", "don't want to be here",
        "nothing matters", "can't take it anymore", "so done", "hate myself"
    ]
    if any(x in t for x in medium_risk_keywords):
        return 1
    return 0

df["risk_id"] = df["clf_text"].apply(infer_risk)
NUM_RISK = 3

print(f"Risk distribution: {df['risk_id']. value_counts().to_dict()}")

PART 1: Data Preprocessing
Dataset size: 61682
Number of emotions: 32
Emotions:  ['afraid', 'angry', 'annoyed', 'anticipating', 'anxious', 'apprehensive', 'ashamed', 'caring', 'confident', 'content', 'devastated', 'disappointed', 'disgusted', 'embarrassed', 'excited', 'faithful', 'furious', 'grateful', 'guilty', 'hopeful', 'impressed', 'jealous', 'joyful', 'lonely', 'nostalgic', 'prepared', 'proud', 'sad', 'sentimental', 'surprised', 'terrified', 'trusting']
Risk distribution: {0: 61620, 1: 36, 2: 26}


In [7]:
print("\n" + "=" * 80)
print("PART 2: Training Mental State Classifier")
print("=" * 80)

tokenizer = AutoTokenizer.from_pretrained(config.CLASSIFIER_MODEL)

class MentalStateDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.texts = df["clf_text"].tolist()
        self.emotion = df["label"].tolist()
        self.intent = df["intent_id"].tolist()
        self.risk = df["risk_id"].tolist()
        self.intensity = df["intensity"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "emotion": torch.tensor(self.emotion[idx], dtype=torch.long),
            "intent": torch.tensor(self.intent[idx], dtype=torch.long),
            "risk": torch.tensor(self.risk[idx], dtype=torch.long),
            "intensity": torch.tensor(self.intensity[idx], dtype=torch.float)
        }

# Stratified split
train_df, val_df = train_test_split(
    df, test_size=0.1, random_state=42, stratify=df["label"]
)

train_ds = MentalStateDataset(train_df, tokenizer, config.CLASSIFIER_MAX_LEN)
val_ds = MentalStateDataset(val_df, tokenizer, config.CLASSIFIER_MAX_LEN)

train_loader = DataLoader(
    train_ds, batch_size=config.CLASSIFIER_BATCH_SIZE, 
    shuffle=True, num_workers=0, pin_memory=True
)
val_loader = DataLoader(
    val_ds, batch_size=config.CLASSIFIER_BATCH_SIZE, 
    num_workers=0, pin_memory=True
)

print(f"Using device: {config.DEVICE}")
print_gpu_memory()

# Initialize model
model = ImprovedMentalStateModel(
    NUM_EMOTIONS, NUM_INTENTS, NUM_RISK, config.CLASSIFIER_MODEL
).to(config.DEVICE)

# Loss functions
loss_emotion = nn.CrossEntropyLoss(label_smoothing=0.1)
loss_intent = nn.CrossEntropyLoss()
risk_weights = torch.tensor([1.0, 3.0, 5.0]).to(config.DEVICE)
loss_risk = nn.CrossEntropyLoss(weight=risk_weights)
loss_intensity = nn.MSELoss()

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=config.CLASSIFIER_LR, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=config.CLASSIFIER_EPOCHS * len(train_loader)
)

def train_epoch(model, loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    correct_emotion = 0
    total = 0
    
    for batch in loader:
        optimizer.zero_grad()
        batch = {k: v.to(config.DEVICE) for k, v in batch.items()}
        out = model(batch["input_ids"], batch["attention_mask"])
        
        loss = (
            1.0 * loss_emotion(out["emotion"], batch["emotion"]) +
            0.5 * loss_intent(out["intent"], batch["intent"]) +
            1.5 * loss_risk(out["risk"], batch["risk"]) +
            0.3 * loss_intensity(out["intensity"], batch["intensity"])
        )
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        correct_emotion += (out["emotion"].argmax(dim=1) == batch["emotion"]).sum().item()
        total += batch["emotion"].size(0)
    
    return total_loss / len(loader), correct_emotion / total

def validate(model, loader):
    model.eval()
    total_loss = 0
    correct_emotion = 0
    correct_risk = 0
    total = 0
    
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(config.DEVICE) for k, v in batch.items()}
            out = model(batch["input_ids"], batch["attention_mask"])
            
            loss = (
                loss_emotion(out["emotion"], batch["emotion"]) +
                0.5 * loss_intent(out["intent"], batch["intent"]) +
                1.5 * loss_risk(out["risk"], batch["risk"]) +
                0.3 * loss_intensity(out["intensity"], batch["intensity"])
            )
            
            total_loss += loss.item()
            correct_emotion += (out["emotion"].argmax(dim=1) == batch["emotion"]).sum().item()
            correct_risk += (out["risk"].argmax(dim=1) == batch["risk"]).sum().item()
            total += batch["emotion"].size(0)
    
    return total_loss / len(loader), correct_emotion / total, correct_risk / total

# Training loop
print("\nTraining Mental State Model...")
best_val_acc = 0

for epoch in range(config.CLASSIFIER_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler)
    val_loss, val_acc, risk_acc = validate(model, val_loader)
    
    print(f"Epoch {epoch+1}/{config.CLASSIFIER_EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:  {val_loss:.4f}, Val Acc: {val_acc:.4f}, Risk Acc: {risk_acc:.4f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'id2label': id2label,
            'label2id': label2id,
            'config': {
                'num_emotions': NUM_EMOTIONS,
                'num_intents': NUM_INTENTS,
                'num_risk': NUM_RISK,
                'model_name': config.CLASSIFIER_MODEL
            }
        }, f"{config.OUTPUT_DIR}/mental_state_model_best.pth")
        print(f"  ✓ Saved best model (Val Acc: {val_acc:.4f})")

print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")


PART 2: Training Mental State Classifier
Using device: cuda
GPU Memory:  0.00GB allocated, 0.00GB reserved

Training Mental State Model...
Epoch 1/5
  Train Loss: 3.0039, Train Acc: 0.2348
  Val Loss:  2.6884, Val Acc: 0.3112, Risk Acc: 0.9992
  ✓ Saved best model (Val Acc: 0.3112)
Epoch 2/5
  Train Loss: 2.5288, Train Acc: 0.3540
  Val Loss:  2.6497, Val Acc: 0.3341, Risk Acc: 0.9994
  ✓ Saved best model (Val Acc: 0.3341)
Epoch 3/5
  Train Loss: 2.2931, Train Acc: 0.4227
  Val Loss:  2.6931, Val Acc: 0.3323, Risk Acc: 0.9994
Epoch 4/5
  Train Loss: 2.1090, Train Acc: 0.4807
  Val Loss:  2.7240, Val Acc: 0.3346, Risk Acc: 0.9994
  ✓ Saved best model (Val Acc: 0.3346)
Epoch 5/5
  Train Loss: 2.0191, Train Acc: 0.5117
  Val Loss:  2.7557, Val Acc: 0.3302, Risk Acc: 0.9994

Best Validation Accuracy: 0.3346


In [8]:
print("\n" + "=" * 80)
print("PART 3:  Generating Policy Training Data")
print("=" * 80)

def policy_label(state):
    """Determine response policy based on mental state."""
    if state["risk"] == 2:
        return "CRISIS_SUPPORT"
    if state["risk"] == 1:
        return "GENTLE_CHECK"
    if state["intent"] == INTENT_MAP["venting"]:
        return "VIBE_CHECK"
    if state["intent"] == INTENT_MAP["advice"]:
        return "REAL_TALK"
    return "HYPE_SESSION"

# Explicitly recreate and load model before inference
print("Loading best classifier model...")
classifier_model = ImprovedMentalStateModel(
    NUM_EMOTIONS, NUM_INTENTS, NUM_RISK, config.CLASSIFIER_MODEL
).to(config.DEVICE)

checkpoint = torch.load(f"{config.OUTPUT_DIR}/mental_state_model_best.pth")
classifier_model.load_state_dict(checkpoint['model_state_dict'])
classifier_model.eval()

# Clean up training model
# del model
clear_gpu_memory()

print("Generating mental state predictions...")
model1_outputs = []

with torch.no_grad():
    for idx, text in enumerate(df["clf_text"]):
        if idx % 500 == 0:
            print(f"Processing {idx}/{len(df)}")
        
        enc = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=config.CLASSIFIER_MAX_LEN,
            return_tensors="pt"
).to(config.DEVICE)
        
        out = classifier_model(enc["input_ids"], enc["attention_mask"])
        
        model1_outputs.append({
            "text": text,
            "emotion": id2label[out["emotion"].argmax().item()],
            "intent":  int(out["intent"].argmax().item()),
            "risk": int(out["risk"].argmax().item()),
            "intensity": float(out["intensity"].item()),
            "confidence": float(out["confidence"].item())
        })

# Clear classifier model from memory
del classifier_model
clear_gpu_memory()
print_gpu_memory()

# System prompt defined separately
SYSTEM_PROMPT = """You're Tuesday, a supportive GenZ mental wellness companion who actually gets it. 

## Response Modes:
- **VIBE_CHECK**: When someone needs to vent.  Validate without fixing.
- **REAL_TALK**: When they want practical advice. Keep it actionable but gentle.
- **HYPE_SESSION**: Default validation and encouragement. 
- **GENTLE_CHECK**: Medium concern situations. Warm, caring, gently suggest resources.  
- **CRISIS_SUPPORT**: Serious situations ONLY. Direct, caring, provide specific resources.

## Style Guide:  
✓ Natural language:  "tbh", "ngl", "lowkey", "honestly", "fr" (don't overuse)
✓ 1-2 emojis max when appropriate 💙
✓ Short paragraphs, conversational flow
✓ Acknowledge feelings before anything else
✗ Never minimize feelings
✗ No toxic positivity"""

# Response templates
import random

RESPONSE_TEMPLATES = {
    "VIBE_CHECK": [
        "I hear you, {emotion} is {intensity_word} to sit with.  It makes total sense you'd feel this way.  {empathy}",
        "Honestly, that sounds really {intensity_word}. Your feelings are valid, and I'm here.  {empathy}",
        "Ngl, that's a lot to carry.  Feeling {emotion} right now is completely understandable. {empathy}",
        "That's {intensity_word}, fr. You don't have to have it all figured out. {empathy}",
    ],
    "REAL_TALK": [
        "I get that you're feeling {emotion}. Here's the thing - {advice}",
        "Okay so, {emotion} is tough.  Lowkey, something that might help:  {advice}",
        "Real talk:  what you're going through is valid.  One thing to consider: {advice}",
        "I hear the {emotion}.  Honestly, {advice}",
    ],
    "HYPE_SESSION": [
        "Can we talk about how you're actually handling this?  {validation} That's growth.",
        "Okay but {validation} The fact that you're even here shows strength ngl.",
        "You're being hard on yourself rn. What I see:  {validation}",
        "Honestly?  {validation} That takes real courage. 💙",
    ],
    "GENTLE_CHECK": [
        "I want you to know I'm hearing you, and what you're feeling matters. {gentle_support}",
        "That sounds really heavy. I'm here with you.  {gentle_support}",
        "I appreciate you sharing this with me. You don't have to go through this alone.  {gentle_support}",
    ],
    "CRISIS_SUPPORT": [
        "I'm really concerned about what you're sharing, and I want you to get support right now. Please reach out to the 988 Suicide & Crisis Lifeline (call or text 988) or Crisis Text Line (text HOME to 741741). You deserve help.  💙",
        "What you're going through sounds incredibly hard. Please reach out to a crisis counselor right now - call/text 988 or text HOME to 741741. You matter, and help is available.",
    ]
}

INTENSITY_WORDS = {
    (0.0, 0.3): "real",
    (0.3, 0.5): "tough",
    (0.5, 0.7): "really hard",
    (0.7, 0.85): "really heavy",
    (0.85, 1.0): "overwhelming"
}

EMPATHY_LINES = [
    "I'm here with you.  💙",
    "You don't have to face this alone.",
    "Take your time, no rush.",
    "That's a lot to hold.",
]

ADVICE_LINES = [
    "Start with one small thing that feels manageable.",
    "What's one tiny step that wouldn't feel overwhelming?",
    "Sometimes just naming it helps.  You've already done that.",
    "Be gentle with yourself while you figure this out.",
]

VALIDATION_LINES = [
    "you're navigating something genuinely difficult.",
    "you're showing up even when it's hard.",
    "you're more capable than you're giving yourself credit for.",
    "the effort you're putting in matters.",
]

GENTLE_SUPPORT_LINES = [
    "If things feel too heavy, talking to someone who specializes in this could really help.  Would you be open to that?",
    "You deserve support through this. Have you considered talking to a counselor?",
    "There are people who want to help.  It's okay to reach out.",
]

def get_intensity_word(intensity):
    for (low, high), word in INTENSITY_WORDS.items():
        if low <= intensity < high:
            return word
    return "real"

# Generate training data
policy_rows = []
intent_names = ["venting", "seeking advice", "looking for validation"]

for s in model1_outputs:
    mode = policy_label(s)
    intensity_word = get_intensity_word(s["intensity"])
    
    # FIXED: Correct f-string formatting (no space)
    user_message = s['text']
    context = f"[{s['emotion']}, intensity:{s['intensity']:.2f}, {intent_names[s['intent']]}, risk:{s['risk']}]"
    
    # Generate response from template
    if mode == "CRISIS_SUPPORT":
        response = random.choice(RESPONSE_TEMPLATES["CRISIS_SUPPORT"])
    else:
        template = random.choice(RESPONSE_TEMPLATES[mode])
        response = template.format(
            emotion=s["emotion"],
            intensity_word=intensity_word,
            empathy=random.choice(EMPATHY_LINES),
            advice=random.choice(ADVICE_LINES),
            validation=random.choice(VALIDATION_LINES),
            gentle_support=random.choice(GENTLE_SUPPORT_LINES)
        )
    
    policy_rows.append({
        "user":  user_message,
        "context": context,
        "mode": mode,
        "response":  response,
        "emotion": s["emotion"]
    })

print(f"Generated {len(policy_rows)} training examples")
print(f"Mode distribution: {pd.Series([r['mode'] for r in policy_rows]).value_counts().to_dict()}")


PART 3:  Generating Policy Training Data
Loading best classifier model...
Generating mental state predictions...
Processing 0/61682
Processing 500/61682
Processing 1000/61682
Processing 1500/61682
Processing 2000/61682
Processing 2500/61682
Processing 3000/61682
Processing 3500/61682
Processing 4000/61682
Processing 4500/61682
Processing 5000/61682
Processing 5500/61682
Processing 6000/61682
Processing 6500/61682
Processing 7000/61682
Processing 7500/61682
Processing 8000/61682
Processing 8500/61682
Processing 9000/61682
Processing 9500/61682
Processing 10000/61682
Processing 10500/61682
Processing 11000/61682
Processing 11500/61682
Processing 12000/61682
Processing 12500/61682
Processing 13000/61682
Processing 13500/61682
Processing 14000/61682
Processing 14500/61682
Processing 15000/61682
Processing 15500/61682
Processing 16000/61682
Processing 16500/61682
Processing 17000/61682
Processing 17500/61682
Processing 18000/61682
Processing 18500/61682
Processing 19000/61682
Processing 19

In [9]:
import inspect
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig

try:
    from trl import SFTTrainer, SFTConfig
except ImportError:
    raise ImportError("TRL is not correctly installed. Please run: pip install trl")

try:
    from trl import DataCollatorForCompletionOnlyLM
except ImportError:
    print("! Could not import DataCollatorForCompletionOnlyLM from trl. Will use custom collator.")
    DataCollatorForCompletionOnlyLM = None


print("\n" + "=" * 80)
print("PART 4: Fine-tuning Phi-3.5 Mini (Optimized for 12GB VRAM)")
print("=" * 80)

clear_gpu_memory()
print_gpu_memory()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Loading Phi-3.5 Mini with 4-bit quantization...")
phi_tokenizer = AutoTokenizer.from_pretrained(
    config.POLICY_MODEL,
    trust_remote_code=True,
)
phi_tokenizer.pad_token = phi_tokenizer.eos_token
phi_tokenizer.padding_side = "right"

try:
    phi_model = AutoModelForCausalLM.from_pretrained(
        config.POLICY_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
    )
    print("✓ Using Flash Attention 2")
except Exception as e:
    print(f"Flash Attention 2 not available, using default attention: {e}")
    phi_model = AutoModelForCausalLM.from_pretrained(
        config.POLICY_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )

phi_model = prepare_model_for_kbit_training(phi_model)
print_gpu_memory()

lora_config = LoraConfig(
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=config.LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

phi_model = get_peft_model(phi_model, lora_config)
phi_model.print_trainable_parameters()
print_gpu_memory()


def format_chat_template(example):
    """Format example using Phi-3's chat template."""
    text = f"""<|system|>
{SYSTEM_PROMPT}<|end|>
<|user|>
{example['user']}<|end|>
<|assistant|>
{example['context']} {example['mode']}

{example['response']}<|end|>"""
    return {"text": text}

hf_dataset = HFDataset.from_list(policy_rows)
hf_dataset = hf_dataset.map(format_chat_template, remove_columns=hf_dataset.column_names)

print("Tokenizing dataset manually...")
def tokenize_function(examples):
    return phi_tokenizer(
        examples["text"],
        truncation=True,
        max_length=config.POLICY_MAX_LEN,
        padding=False,
    )

tokenized_dataset = hf_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"] 
)

training_args_kwargs = dict(
    output_dir=f"{config.OUTPUT_DIR}/phi35_genz_therapist",
    per_device_train_batch_size=config.POLICY_BATCH_SIZE,
    gradient_accumulation_steps=config.POLICY_GRAD_ACCUM,
    learning_rate=config.POLICY_LR,
    num_train_epochs=config.POLICY_EPOCHS,
    fp16=False,
    bf16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=25,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    dataloader_num_workers=0,
    remove_unused_columns=False,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
)

sft_signature = inspect.signature(SFTTrainer.__init__)
use_legacy_trl = ("tokenizer" in sft_signature.parameters) and (SFTConfig is None)

if use_legacy_trl:
    training_args = TrainingArguments(**training_args_kwargs)
else:
    sft_config = SFTConfig(
        **training_args_kwargs,
        dataset_text_field=None, 
        max_length=config.POLICY_MAX_LEN,
        packing=False,
    )

if DataCollatorForCompletionOnlyLM is not None:
    print("✓ Using trl's DataCollatorForCompletionOnlyLM")
    collator = DataCollatorForCompletionOnlyLM(
        response_template="<|assistant|>",
        tokenizer=phi_tokenizer,
        mlm=False,
        pad_to_multiple_of=8,
    )
else:
    print("✓ Using custom CustomCompletionOnlyCollator")
    collator = CustomCompletionOnlyCollator(
        tokenizer=phi_tokenizer,
        response_template="<|assistant|>",
        mlm=False,
    )

if use_legacy_trl:
    trainer = SFTTrainer(
        model=phi_model,
        train_dataset=tokenized_dataset, 
        args=training_args,
        tokenizer=phi_tokenizer,
        dataset_text_field=None, 
        max_seq_length=config.POLICY_MAX_LEN,
        data_collator=collator,
        packing=False,
    )
else:
    trainer = SFTTrainer(
        model=phi_model,
        train_dataset=tokenized_dataset,
        args=sft_config,
        processing_class=phi_tokenizer,
        data_collator=collator,
    )

print("\nTraining Phi-3.5 GenZ Therapist...")
print_gpu_memory()

trainer.train()

print("\nSaving model...")
trainer.save_model(f"{config.OUTPUT_DIR}/phi35_genz_therapist_final")
phi_tokenizer.save_pretrained(f"{config.OUTPUT_DIR}/phi35_genz_therapist_final")

with open(f"{config.OUTPUT_DIR}/phi35_genz_therapist_final/system_prompt.txt", "w", encoding="utf-8") as f:
    f.write(SYSTEM_PROMPT)

! Could not import DataCollatorForCompletionOnlyLM from trl. Will use custom collator.

PART 4: Fine-tuning Phi-3.5 Mini (Optimized for 12GB VRAM)
GPU Memory:  1.40GB allocated, 1.61GB reserved
Loading Phi-3.5 Mini with 4-bit quantization...


`torch_dtype` is deprecated! Use `dtype` instead!
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Flash Attention 2 not available, using default attention: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.52s/it]


GPU Memory:  4.06GB allocated, 5.28GB reserved
trainable params: 8,912,896 || all params: 3,829,992,448 || trainable%: 0.2327
GPU Memory:  4.09GB allocated, 5.32GB reserved


...

Truncating train dataset: 100%|██████████| 61682/61682 [00:00<00:00, 670960.72 examples/s]


Training Phi-3.5 GenZ Therapist...
GPU Memory:  4.08GB allocated, 5.32GB reserved



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
25,4.602600
50,4.445600
75,4.106600
100,3.326500
125,2.416300
150,1.449200
175,0.891000
200,0.536500
225,0.340500
250,0.251100



Saving model...


In [10]:
print("\n" + "=" * 80)
print("PART 5: Creating Unified Inference Pipeline")
print("=" * 80)

inference_code = '''"""
Tuesday Bot - GenZ Mental Wellness Companion
Inference Pipeline
"""
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch.nn as nn
import torch.nn.functional as F
import os


class ImprovedMentalStateModel(nn.Module):
    """Multi-task mental state classifier."""
    
    def __init__(self, num_emotions, num_intents, num_risk, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        h = self.encoder.config.hidden_size
        
        self.shared1 = nn.Linear(h, h)
        self.shared2 = nn.Linear(h, h)
        self.layer_norm = nn.LayerNorm(h)
        self.dropout = nn.Dropout(0.3)
        
        self.emotion_hidden = nn.Linear(h, h // 2)
        self.emotion_head = nn.Linear(h // 2, num_emotions)
        self.intent_hidden = nn.Linear(h, h // 4)
        self.intent_head = nn.Linear(h // 4, num_intents)
        self.risk_hidden = nn.Linear(h, h // 4)
        self.risk_head = nn.Linear(h // 4, num_risk)
        self.intensity_hidden = nn.Linear(h, h // 4)
        self.intensity_head = nn.Linear(h // 4, 1)
        self.confidence_head = nn.Linear(h, 1)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids, attention_mask=attention_mask)
        hidden = out.last_hidden_state
        attention_weights = attention_mask.unsqueeze(-1).float()
        pooled = (hidden * attention_weights).sum(dim=1) / attention_weights.sum(dim=1).clamp(min=1e-9)
        
        z = self.dropout(F.gelu(self.shared1(pooled)))
        z = self.layer_norm(z + self.dropout(F.gelu(self.shared2(z))))
        
        emotion_h = self.dropout(F.gelu(self.emotion_hidden(z)))
        intent_h = self.dropout(F.gelu(self.intent_hidden(z)))
        risk_h = self.dropout(F.gelu(self.risk_hidden(z)))
        intensity_h = self.dropout(F.gelu(self.intensity_hidden(z)))
        
        return {
            "emotion": self.emotion_head(emotion_h),
            "intent": self.intent_head(intent_h),
            "risk": self.risk_head(risk_h),
            "intensity": torch.sigmoid(self.intensity_head(intensity_h)).squeeze(-1),
            "confidence": torch.sigmoid(self.confidence_head(z)).squeeze(-1),
        }


class TuesdayBot:
    """Unified inference pipeline for the GenZ Therapist Bot."""
    
    def __init__(self, models_dir="./models", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.models_dir = models_dir
        
        # Load system prompt
        prompt_path = f"{models_dir}/phi35_genz_therapist_final/system_prompt.txt"
        if os.path.exists(prompt_path):
            with open(prompt_path, "r", encoding="utf-8") as f:
                self.system_prompt = f.read()
        else:
            self.system_prompt = "You're Tuesday, a supportive GenZ mental wellness companion."
        
        self._load_classifier()
        self._load_generator()
        
        self.intent_names = ["venting", "seeking advice", "looking for validation"]
        
    def _load_classifier(self):
        """Load the mental state classification model."""
        print("Loading mental state classifier...")
        checkpoint = torch.load(
            f"{self.models_dir}/mental_state_model_best.pth",
            map_location=self.device
        )
        cfg = checkpoint['config']
        
        self.classifier_tokenizer = AutoTokenizer.from_pretrained(cfg['model_name'])
        
        self.classifier = ImprovedMentalStateModel(
            cfg['num_emotions'], cfg['num_intents'], cfg['num_risk'], cfg['model_name']
        )
        self.classifier.load_state_dict(checkpoint['model_state_dict'])
        self.classifier.to(self.device)
        self.classifier.eval()
        
        self.id2label = checkpoint['id2label']
        print(f"  ✓ Loaded classifier with {cfg['num_emotions']} emotions")
        
    def _load_generator(self):
        """Load the fine-tuned response generator."""
        print("Loading response generator...")
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True
        )
        
        model_path = f"{self.models_dir}/phi35_genz_therapist_final"
        
        self.gen_tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.gen_tokenizer.pad_token = self.gen_tokenizer.eos_token
        
        base_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/Phi-3.5-mini-instruct",
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        )
        
        self.generator = PeftModel.from_pretrained(base_model, model_path)
        self.generator.eval()
        print("  ✓ Loaded response generator")
    
    def analyze_mental_state(self, text):
        """Analyze the mental state from user input."""
        enc = self.classifier_tokenizer(
            text.lower().strip(),
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            out = self.classifier(enc["input_ids"], enc["attention_mask"])
        
        return {
            "emotion": self.id2label[out["emotion"].argmax().item()],
            "intent": int(out["intent"].argmax().item()),
            "risk": int(out["risk"].argmax().item()),
            "intensity": float(out["intensity"].item()),
            "confidence": float(out["confidence"].item())
        }
    
    def get_response_mode(self, state):
        """Determine the appropriate response mode."""
        if state["risk"] == 2:
            return "CRISIS_SUPPORT"
        if state["risk"] == 1:
            return "GENTLE_CHECK"
        if state["intent"] == 0:
            return "VIBE_CHECK"
        if state["intent"] == 1:
            return "REAL_TALK"
        return "HYPE_SESSION"
    
    def generate_response(self, user_input, max_new_tokens=256, temperature=0.7):
        """Generate a supportive response."""
        state = self.analyze_mental_state(user_input)
        mode = self.get_response_mode(state)
        
        # Build prompt in Phi-3 format
        prompt = f"""<|system|>
{self.system_prompt}<|end|>
<|user|>
{user_input}<|end|>
<|assistant|>
"""
        
        # Add context for model (FIXED: correct f-string format)
        context = f"[{state['emotion']}, intensity:{state['intensity']:.2f}, {self.intent_names[state['intent']]}, risk:{state['risk']}] {mode}\\n\\n"
        prompt = prompt + context
        
        inputs = self.gen_tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.generator.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.gen_tokenizer.eos_token_id
            )
        
        full_response = self.gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract just the generated part
        if "<|assistant|>" in full_response:
            response = full_response.split("<|assistant|>")[-1]
        else:
            response = full_response
        
        response = response.replace("<|end|>", "").strip()
        
        # Remove context prefix if present
        if response.startswith("["):
            bracket_end = response.find("]")
            if bracket_end != -1:
                response = response[bracket_end + 1:].strip()
                for m in ["CRISIS_SUPPORT", "GENTLE_CHECK", "VIBE_CHECK", "REAL_TALK", "HYPE_SESSION"]:
                    if response.startswith(m):
                        response = response[len(m):].strip()
        
        return {
            "response": response,
            "mental_state": state,
            "mode": mode
        }
    
    def chat(self):
        """Interactive chat mode."""
        print("\\n" + "=" * 60)
        print("Tuesday Bot - GenZ Mental Wellness Companion")
        print("Type 'quit' to exit")
        print("=" * 60 + "\\n")
        
        while True:
            user_input = input("You: ").strip()
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\\nTake care! Remember, you're doing better than you think.  💙")
                break
            
            if not user_input:
                continue
            
            result = self.generate_response(user_input)
            print(f"\\n[{result['mode']}] Tuesday:  {result['response']}\\n")


if __name__ == "__main__": 
    bot = TuesdayBot()
    
    test_inputs = [
        "I've been feeling so anxious about my job interview tomorrow",
        "Just had the best day ever!  Got promoted! ",
        "I feel like nobody understands me and I'm so alone",
        "What should I do about my relationship problems?",
    ]
    
    print("\\n" + "=" * 60)
    print("Testing Tuesday Bot")
    print("=" * 60)
    
    for text in test_inputs:
        print(f"\\nUser: {text}")
        result = bot.generate_response(text)
        # FIXED: Correct f-string format
        print(f"State: {result['mental_state']['emotion']} (intensity: {result['mental_state']['intensity']:.2f})")
        print(f"Mode: {result['mode']}")
        print(f"Tuesday:  {result['response']}")
        print("-" * 50)
    
    print("\\nStarting interactive chat...")
    bot.chat()
'''

with open(f"{config.OUTPUT_DIR}/tuesday_bot.py", "w", encoding="utf-8") as f:
    f.write(inference_code)

print(f"✓ Saved inference pipeline to {config.OUTPUT_DIR}/tuesday_bot.py")


PART 5: Creating Unified Inference Pipeline
✓ Saved inference pipeline to ./models/tuesday_bot.py


In [11]:
print("\n" + "=" * 80)
print("✨ PIPELINE COMPLETE!  ✨")
print("=" * 80)
print(f"""
Models trained and saved to {config.OUTPUT_DIR}/: 
1. mental_state_model_best.pth - Multi-task emotion/intent/risk classifier
2. phi35_genz_therapist_final/ - Fine-tuned Phi-3.5 Mini response generator

Bugs Fixed: 
✓ File path:  Removed space in "cleaned. csv"
✓ Float formatting: Fixed all ":. 2f" syntax (removed spaces)
✓ Model scope: ImprovedMentalStateModel defined globally
✓ System prompt: Using manual Phi-3 format
✓ DataCollator: Custom implementation for older trl versions

Memory Optimizations for RTX 4080 12GB:
✓ 4-bit quantization with double quantization
✓ bfloat16 compute dtype
✓ Gradient checkpointing
✓ 8-bit paged AdamW optimizer
✓ Flash Attention 2 (if available)

Usage:
    from tuesday_bot import TuesdayBot
    bot = TuesdayBot(models_dir="{config.OUTPUT_DIR}")
    result = bot.generate_response("I'm feeling anxious today")
    print(result["response"])
    
    # Or interactive chat: 
    bot.chat()
""")

print_gpu_memory()


✨ PIPELINE COMPLETE!  ✨

Models trained and saved to ./models/: 
1. mental_state_model_best.pth - Multi-task emotion/intent/risk classifier
2. phi35_genz_therapist_final/ - Fine-tuned Phi-3.5 Mini response generator

Bugs Fixed: 
✓ File path:  Removed space in "cleaned. csv"
✓ Float formatting: Fixed all ":. 2f" syntax (removed spaces)
✓ Model scope: ImprovedMentalStateModel defined globally
✓ System prompt: Using manual Phi-3 format
✓ DataCollator: Custom implementation for older trl versions

Memory Optimizations for RTX 4080 12GB:
✓ 4-bit quantization with double quantization
✓ bfloat16 compute dtype
✓ Gradient checkpointing
✓ 8-bit paged AdamW optimizer
✓ Flash Attention 2 (if available)

Usage:
    from tuesday_bot import TuesdayBot
    bot = TuesdayBot(models_dir="./models")
    result = bot.generate_response("I'm feeling anxious today")
    print(result["response"])

    # Or interactive chat: 
    bot.chat()

GPU Memory:  4.09GB allocated, 5.34GB reserved
